<a href="https://colab.research.google.com/github/joker2017/Calculator/blob/master/12122018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from functools import partial
import tensorflow as tf
import numpy as np
import os

tf.reset_default_graph()

learning_rate = 0.01
batch_size = 1000
n_layers_1 = 300
n_layers_2 = 200
n_layers_3 = 100
n_layers_4 = 50
epoch_max = 100
output = 10


X = tf.placeholder (tf.float32, shape=(None, 28*28), name="X")
y = tf.placeholder (tf.int32, shape=(None), name="y")
training = tf.placeholder_with_default(False, shape=(None), name="training")
batch_norm_layer = partial(tf.layers.batch_normalization, training=training, momentum=0.9)

def shuffle_batch (X, y, batch_size):
  rnd_id = np.randome.permutation(len(X))
  batch_rnd = len(X)//batch_size
  for batch_n in np.array_split(rnd_id, batch_rnd):
    X_batch, y_batch = X[batch_n], y[batch_n]
  yield X_batch, y_batch
  
(X_train, y_train), (X_valid, y_valid) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28)
X_valid = X_valid.astype(np.float32).reshape(-1, 28*28)
y_train = y_train.astype(np.int32)
y_valid = y_valid.astype(np.int32)
X_train, X_valid = X_train[:5000], X_train[5000:]
y_train, y_valid = y_train[:5000], y_train[5000:]

with tf.name_scope("DNN"):
  
  hidden_1 = tf.layers.dense(X, n_layers_1, activation=tf.nn.elu, name="hidden_1")
  
  hidden_2 = tf.layers.dense(hidden_1, n_layers_2, name="hidden_2")
  hidden_2_bn = batch_norm_layer(hidden_2, name="hidden_2_bn")
  Hidden_2_bn_act = tf.nn.relu(hidden_2_bn, name="Hidden_2_bn_act")
  
  hidden_3 = tf.layers.dense(Hidden_2_bn_act, n_layers_3, name="hidden_3")
  hidden_3_bn = batch_norm_layer(hidden_3, name="hidden_3_bn")
  Hidden_3_bn_act = tf.nn.relu(hidden_3_bn, name="Hidden_3_bn_act")
  
  hidden_4 = tf.layers.dense(Hidden_3_bn_act, n_layers_4, name="hidden_4")
  hidden_4_bn = batch_norm_layer(hidden_4, name="hidden_4_bn")
  Hidden_4_bn_act = tf.nn.relu(hidden_4_bn, name="Hidden_4_bn_act") 
  
  logits_without_bn = tf.layers.dense(Hidden_4_bn_act, output, name="logits_without_bn")
  logits = batch_norm_layer(logits_without_bn, name="logits")
  
with tf.name_scope("loss"):
  xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits=logits)
  loss = tf.reduce_mean(xentropy)
  loss_sum = tf.summary.scalar('loss_sum', loss)
  
with tf.name_scope("train"):
  optimizer = tf.train.AdamOptimizer(learning_rate)
  training_op = optimizer.minimize(loss)
  
with tf.name_scope("eval"):
  prediction = tf.nn.in_top_k(logits, y, 1)
  accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
  accuracy_sum = tf.summary.scalar('accuracy_sum', accuracy)
  
  